<a href="https://colab.research.google.com/github/AllyHyeseongKim/2020_cau_oss_hackathon/blob/master/hackathon_team06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 5시 (단, 발표자료는 11시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2020_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [54]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

# 데이터셋 다운로드
check = !if [ -d 'dataset/' ]; then echo "1" ; else echo "0"; fi
if (check[0] is '0' ):
  !mkdir dataset
  !wget 'https://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/matlab.zip'
  !unzip matlab.zip -d /content/dataset

# 데이터셋 로드
from scipy import io as spio
emnist = spio.loadmat("/content/dataset/matlab/emnist-balanced.mat")

x_train = emnist["dataset"][0][0][0][0][0][0]
y_train = emnist["dataset"][0][0][0][0][0][1]

x_test = emnist["dataset"][0][0][1][0][0][0]
y_test = emnist["dataset"][0][0][1][0][0][1]

# # 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 데이터 28x28 이미지화
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

# 총 클래스 개수
num_classes = y_test.shape[1]
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**



In [55]:
tf.debugging.set_log_device_placement(True)

In [56]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

# **3. 모델 생성**

In [ ]:
'''
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Flatten layer: 28 x 28 x 1 image를 784개의 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=input_shape))

# 1st hidden layer: fully-connected layer
# (# of inputs = 784, # of outputs = 512, actication fuction = relu)
model.add(keras.layers.Dense(512, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 512, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(64, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 47, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 5, shuffle=True, callbacks=[cp_callback], validation_data=(x_test_after, y_test))
'''

Epoch 1/5
865/882 [============================>.] - ETA: 0s - loss: 0.9878 - accuracy: 0.7078
Epoch 00001: val_accuracy improved from -inf to 0.79580, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 3s 3ms/step - loss: 0.9813 - accuracy: 0.7095 - val_loss: 0.6317 - val_accuracy: 0.7958
Epoch 2/5
869/882 [============================>.] - ETA: 0s - loss: 0.5308 - accuracy: 0.8235
Epoch 00002: val_accuracy improved from 0.79580 to 0.82564, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 3s 3ms/step - loss: 0.5300 - accuracy: 0.8237 - val_loss: 0.5331 - val_accuracy: 0.8256
Epoch 3/5
873/882 [============================>.] - ETA: 0s - loss: 0.4430 - accuracy: 0.8468
Epoch 00003: val_accuracy improved from 0.82564 to 0.83979, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 3s 3ms/step - loss: 0.4431 - accuracy: 0.8466 - val_loss: 0.4976 - val_accuracy:

In [63]:
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Conv1
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu, input_shape=(np.shape(x_train_after[0]))))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.5))
# Conv2
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.5))
# Conv3
model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.5))
# Conv4
model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", activation=tf.nn.relu))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Flatten())
# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 47, actication fuction = softmax)
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1, min_lr=1e-4)

model.summary()
# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 17, shuffle=True, callbacks=[reduce_lr, cp_callback], validation_data=(x_test_after, y_test))

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_220 (Conv2D)          (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_220 (Bat (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_221 (Conv2D)          (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_221 (Bat (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_88 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_222 (Conv2D)          (None, 14, 14, 128)     

# **4. 모델 저장**

In [66]:
save_path = '/content/'
team_name = 'team06'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [67]:
save_path = '/content/'
team_name = 'team06'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test_after, y_test)

588/588 [==============================] - 2s 4ms/step - loss: 0.2967 - accuracy: 0.9057


[0.29672276973724365, 0.9056915044784546]